**Lesson exercise notebook for Spark-for-Machine-Learning-AI Course**
* Created by Kevin Chao (kevinchao@gmail.com)
* Latest updated on Feb 14, 2024

In [6]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
# Re-create iviris_df
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

iris_df =spark.read.csv("Exercise_Files/iris.txt", inferSchema=True)

iris_df = iris_df.select(
    col("_c0").alias("sepal_length"),
    col("_c1").alias("sepal_width"),
    col("_c2").alias("petal_length"),
    col("_c3").alias("petal_width"),
    col("_c4").alias("species")
    )

vectorAssembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
viris_df = vectorAssembler.transform(iris_df)
indexer = StringIndexer(inputCol="species", outputCol="label")
iviris_df = indexer.fit(viris_df).transform(viris_df)

In [4]:
iviris_df

DataFrame[sepal_length: double, sepal_width: double, petal_length: double, petal_width: double, species: string, features: vector, label: double]

In [5]:
iviris_df.take(1)

[Row(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0)]

In [9]:
splits = iviris_df.randomSplit([0.6, 0.4], 1)  # Split to two groups: 60% and 40%; seed = 1
train_df = splits[0]
test_df = splits[1]

print(iviris_df.count())
print(train_df.count())
print(test_df.count())

150
98
52


In [8]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [14]:
dt_model = dt.fit(train_df)

In [16]:
dt_predictions = dt_model.transform(test_df)

In [17]:
dt_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [18]:
dt_accuracy = dt_evaluator.evaluate(dt_predictions)

In [19]:
dt_accuracy

0.9423076923076923